In [21]:
# load dataset
import dataloader
import utilities
import transcribe
import madmom
import os
import features
import numpy as np
from tqdm import tqdm

parameters = {
        "chroma_type":"crp",  # cqt,crp
        "vocabulary":"majmin",  # majmin, triads, triads_extended or majmin_sevenths
        "eval_scheme":"majmin",
        "source_separation":"demucs", # hprs, demucs or None
        "prefilter":"rp", # None, "median"#
        "embedding":5,
        "neighbours":15,
        "prefilter_length":12,  # N 
        "postfilter":"hmm", # None, "hmm" or "median"
        "transition_prob":0.2,  # 0..1
        "postfilter_length":1, # N
        "dataset":"beatles"  # rwc_popular
}

PATH = "/home/max/ET-TI/Masterarbeit/mirdata/"
output_path = "/home/max/ET-TI/Masterarbeit/results/"
results= []
print(f"Starting Transcription!")
for fold in range(1,9):
    fold_score = 0.0
    fold_seg_score = 0.0
    dataset = dataloader.MIRDataset(parameters.get("dataset","beatles"),basepath=PATH,split_nr=fold)
    for track_id in tqdm(dataset.getTrackList(),desc=f"fold {fold}/8"):
        name = dataset.getTitle(track_id).split("_-_")[-1]
        audiopath,(ref_intervals,ref_labels) = dataset[track_id]

        ## HPS ##
        if parameters.get("source_separation",False) == "hprs":
            time_vector,signal = utilities.loadAudio(audiopath)
            y_harm,y_perc,y_res = transcribe.harmonicPercussiveResidualSeparation(signal,beta=3,n_fft=4096)
            signal = madmom.audio.signal.Signal(y_harm, sample_rate=signal.sample_rate, num_channels=1, start=signal.start, stop=signal.stop)
            rms = features.rms(signal,hop_length=4096)
        elif parameters.get("source_separation",False) == "demucs":
            basepath,filename = os.path.split(audiopath)
            filename = filename.rsplit('.', 1)[0]
            audiopath = basepath+"/instrumentals/"+filename+"_instrumental.mp3"
            time_vector,signal = utilities.loadAudio(audiopath)
            rms = features.rms(signal,hop_length=4096)
        else:
            time_vector,signal = utilities.loadAudio(audiopath)
            rms = features.rms(signal,hop_length=4096)

        if parameters.get("chroma_type","crp") == "crp":    
            t_chroma,chroma = features.crpChroma(signal)
        elif parameters.get("chroma_type") == "dcp":
            t_chroma,chroma = features.deepChroma(signal,split_nr=fold)
        else:
            t_chroma,chroma = features.cqtChroma(signal)
        est_intervals, est_labels = transcribe.transcribeChromagram(t_chroma,chroma,rms,**parameters)
        score,seg_score = transcribe.evaluateTranscription(est_intervals,est_labels,ref_intervals,ref_labels,parameters.get("eval_scheme","majmin"))
        results.append((name,score,seg_score))

mean_score = np.mean(np.array([x[1] for x in results]))
mean_seg_score = np.mean(np.array([x[2] for x in results]))
print(f"{parameters.get('eval_scheme','majmin')}: {round(mean_score,2)}, segmentation: {round(mean_seg_score,2)}")


Starting Transcription!


fold 8/8: 100%|██████████| 23/23 [14:41<00:00, 38.34s/it]

majmin: 0.69, segmentation: 0.74


CRP, N=10, p=0.2 : majmin: 0.69, segmentation: 0.75
CRP, N=12, p=0.15 : majmin: 0.69, segmentation: 0.75
CRP, N=15, p=0.15 : majmin: 0.69, segmentation: 0.74 
DEMUCS, CRP : majmin: 0.72, segmentation: 0.76
DEMUCS, CRP RP(5,15) majmin: 0.69, segmentation 0.74
HPRS, CRP: majmin: 0.7 , segmentation: 0.75


Reference Model: Deep chroma processor
majmin: 0.84, segmentation: 0.89

In [18]:
import numpy as np
results = []
for fold in range(1,9):
    dataset = dataloader.MIRDataset(parameters.get("dataset","beatles"),basepath=PATH,split_nr=fold)
    for track_id in tqdm(dataset.getTrackList(),desc=f"fold {fold}/8"):
        name = dataset.getTitle(track_id).split("_-_")[-1]
        audiopath,(ref_intervals,ref_labels) = dataset[track_id]
        time_vector,signal = utilities.loadAudio(audiopath)
        t_chroma,chroma = features.deepChroma(signal,split_nr=fold)
        est_intervals,est_labels = transcribe.transcribeDeepChroma(t_chroma,chroma)
        score,seg_score = transcribe.evaluateTranscription(est_intervals,est_labels,ref_intervals,ref_labels,parameters.get("eval_scheme","majmin"))
        results.append((name,score,seg_score))

mean_score = np.mean(np.array([x[1] for x in results]))
mean_seg_score = np.mean(np.array([x[2] for x in results]))
print(f"{parameters.get('eval_scheme','majmin')}: {round(mean_score,2)}, segmentation: {round(mean_seg_score,2)}")


fold 8/8: 100%|██████████| 23/23 [00:33<00:00,  1.44s/it]

parameters.get('eval_scheme','majmin'): 0.84, segmentation: 0.89
